In [1]:
# 결과 저장
df_out = df[["title", "single_drugs_str"]]
df_out.to_csv("gold_single.csv", index=False, encoding="utf-8")

print(df_out.head())

NameError: name 'df' is not defined

단일명_정답셋 불러오기

In [7]:
import pandas as pd

# CSV 불러오기
# df = pd.read_csv("/data1/workspace/gold_standardized.csv", encoding="utf-8")  # 필요시 cp949, euc-kr 등으로 변경

# 수정된 코드
df = pd.read_csv('/data1/workspace/gold_standardized.csv', encoding='cp949')
# 또는
# df = pd.read_csv('파일명.csv', encoding='euc-kr')


In [8]:
print(df.head())

                                               title  \
0  conditionally reprogrammed colorectal cancer c...   
1  combining mek and src inhibitors for treatment...   
2  sensitivity of kras-mutant colorectal cancers ...   
3  targeting ras mutant colorectal cancer with du...   
4  mcl-1 inhibition overcomes intrinsic and acqui...   

                                     single_drug  \
0  cetuximab; palbociclib; sorafenib; trametinib   
1                          dasatinib; trametinib   
2                        palbociclib; trametinib   
3                        palbociclib; trametinib   
4                            regorafenib; s63845   

                                 generic_drug  
0  cetuximab;palbociclib;sorafenib;trametinib  
1                        dasatinib;trametinib  
2                      palbociclib;trametinib  
3                      palbociclib;trametinib  
4                          regorafenib;s63845  


비교셋 불러오기

In [9]:
# CSV 불러오기
df1 = pd.read_csv("pred_4.csv", encoding="utf-8")  # 필요시 cp949, euc-kr 등으로 변경

print(df1.head())

                                               drugs  \
0                                             PD;LEE   
1                                                  -   
2                    trametinib; navitoclax; KRT-232   
3                     BGB-283; dabrafenib; cetuximab   
4  lapatinib; trastuzumab; pertuzumab; T-DM1; TAS...   

                                               title  
0  CDK4/6 Inhibitors Impede Chemoresistance and I...  
1  Conditionally reprogrammed colorectal cancer c...  
2  KRT-232 and navitoclax enhance trametinib’s an...  
3  BGB-283, a Novel RAF Kinase and EGFR Inhibitor...  
4  Acquired resistance to trastuzumab/pertuzumab ...  


정답셋, 비교셋 전처리 : 소문자, 특수문자 등 제거

In [15]:
import re
import pandas as pd

def normalize_drug_name(name: str) -> str:
    if pd.isna(name):
        return ""
    name = name.lower()
    name = re.sub(r"[ \-\?\!,]", "", name)
    return name

def preprocess_csv(path_in, path_out):
    # 여러 인코딩 자동 시도
    encodings = ['utf-8', 'cp949', 'euc-kr', 'latin1']
    df = None
    
    for enc in encodings:
        try:
            df = pd.read_csv(path_in, encoding=enc)
            print(f"✓ {path_in}: {enc} 인코딩으로 읽기 성공")
            break
        except (UnicodeDecodeError, UnicodeError):
            continue
    
    if df is None:
        raise ValueError(f"❌ {path_in}의 인코딩을 찾을 수 없습니다.")
    
    if "single_drug" in df.columns:
        df["single_drug_norm"] = df["single_drug"].fillna("").apply(
            lambda x: ";".join([normalize_drug_name(d) for d in x.split(";") if d.strip()])
        )
    else:
        raise ValueError("CSV에 'single_drug' 컬럼이 없습니다.")
    
    df.to_csv(path_out, index=False, encoding="utf-8-sig")
    print(f">>> 저장 완료: {path_out}\n")

# 실행 (각 파일의 인코딩 자동 감지)
preprocess_csv("gold_standardized.csv", "gold_standardized_norm.csv")
preprocess_csv("pred_4.csv", "pred_4_norm.csv")

✓ gold_standardized.csv: cp949 인코딩으로 읽기 성공
>>> 저장 완료: gold_standardized_norm.csv

✓ pred_4.csv: utf-8 인코딩으로 읽기 성공
>>> 저장 완료: pred_4_norm.csv



3가지 지표 계산 : precision, recall, f1-score

In [ ]:
import pandas as pd

# CSV 불러오기
gold = pd.read_csv("gold_standardized_norm.csv")
pred = pd.read_csv("pred_4_norm.csv")

# title 기준 merge
merged = pd.merge(gold, pred, on="title", suffixes=("_gold", "_pred"))

TP, FP, FN = 0, 0, 0

for _, row in merged.iterrows():
    gold_set = set([d for d in str(row["single_drug"]).split(";") if d])
    pred_set = set([d for d in str(row["drugs"]).split(";") if d])

    TP += len(gold_set & pred_set)     # 교집합
    FN += len(gold_set - pred_set)     # 정답에만 있는 경우
    FP += len(pred_set - gold_set)     # 예측에만 있는 경우

precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
recall    = TP / (TP + FN) if (TP + FN) > 0 else 0.0
f1        = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

print("Precision :", precision)
print("Recall    :", recall)
print("F1 Score  :", f1)


UnicodeDecodeError: 'cp949' codec can't decode byte 0xbe in position 7497: illegal multibyte sequence

In [16]:
print(gold.columns)
print(pred.columns)

Index(['title', 'single_drugs_str'], dtype='object')
Index(['drugs', 'title'], dtype='object')


pre,recall,f1 계산

In [12]:
import pandas as pd
import numpy as np

def parse_and_clean_set(text, delimiter=';'):
    """
    문자열을 세트(Set)로 변환하고, 공백 제거 및 소문자 변환을 수행합니다.
    """
    if pd.isna(text) or text.strip() == '':
        return set()
    
    # 쉼표(,)가 아닌 세미콜론(;)을 구분자로 사용
    items = text.split(delimiter)
    
    # 각 항목의 공백 제거 및 소문자 변환 후 Set 반환
    return set(item.strip().lower() for item in items if item.strip())

def calculate_set_metrics(gold, pred, merge_col, gold_col, pred_col):
    """
    두 파일을 병합하고 세트 기반의 Precision, Recall, F1-score를 계산합니다.
    """
    try:
        # 1. 파일 로드
        df_gold = gold
        df_pred = pred

        # 2. 공통열(title)로 데이터프레임 병합
        merged_df = pd.merge(df_gold, df_pred, on=merge_col, how='inner')

        if merged_df.empty:
            print("경고: 병합된 데이터가 없습니다. 공통 'title' 열을 확인해 주세요.")
            return None

        # 3. 항목을 세트(Set)로 변환
        merged_df['gold_set'] = merged_df[gold_col].apply(parse_and_clean_set)
        merged_df['pred_set'] = merged_df[pred_col].apply(parse_and_clean_set)

        # 4. 각 행별 TP, FP, FN 계산 함수
        def calculate_row_metrics(row):
            gt = row['gold_set']
            pred = row['pred_set']
            
            # True Positives: 예측과 정답의 교집합
            tp = len(gt.intersection(pred))
            
            # False Positives: 예측은 했지만 정답이 아닌 것 (예측 - 정답)
            fp = len(pred.difference(gt))
            
            # False Negatives: 정답이지만 예측하지 못한 것 (정답 - 예측)
            fn = len(gt.difference(pred))
            
            return pd.Series({'TP': tp, 'FP': fp, 'FN': fn})

        # 5. 모든 행에 걸쳐 TP, FP, FN 계산
        metrics_df = merged_df.apply(calculate_row_metrics, axis=1)
        
        # 6. Micro-averaged 합계 계산
        total_tp = metrics_df['TP'].sum()
        total_fp = metrics_df['FP'].sum()
        total_fn = metrics_df['FN'].sum()

        # 7. Precision, Recall, F1-score 계산
        
        # 분모가 0인 경우를 처리합니다.
        
        # Precision = TP / (TP + FP)
        if (total_tp + total_fp) == 0:
            precision = 0.0
        else:
            precision = total_tp / (total_tp + total_fp)
        
        # Recall = TP / (TP + FN)
        if (total_tp + total_fn) == 0:
            recall = 0.0
        else:
            recall = total_tp / (total_tp + total_fn)
            
        # F1-score = 2 * (P * R) / (P + R)
        if (precision + recall) == 0:
            f1_score = 0.0
        else:
            f1_score = 2 * (precision * recall) / (precision + recall)

        # 8. 결과 출력
        print(f"**총 비교 대상 행 수:** {len(merged_df)}개")
        print("--- 총 누적 합계 (Micro-averaged Basis) ---")
        print(f"True Positives (TP): {total_tp}")
        print(f"False Positives (FP): {total_fp}")
        print(f"False Negatives (FN): {total_fn}")
        print("------------------------------------------")
        print(f"**Precision (정밀도):** {precision:.4f} ({precision * 100:.2f}%)")
        print(f"**Recall (재현율):** {recall:.4f} ({recall * 100:.2f}%)")
        print(f"**F1-score:** {f1_score:.4f} ({f1_score * 100:.2f}%)")

        return {
            'precision': precision, 
            'recall': recall, 
            'f1_score': f1_score
        }

    except FileNotFoundError as e:
        print(f"오류: 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요: {e}")
        return None
    except KeyError as e:
        print(f"오류: 열 이름을 찾을 수 없습니다. 'title', 'drug_single_str', 'drugs' 열 이름을 확인해 주세요: {e}")
        return None

# --- 실행 부분 (사용 예시) ---
# precision_score = calculate_set_metrics(
#     ground_truth_file='ground_truth.csv',
#     prediction_file='my_predictions.csv',
#     merge_col='title',
#     gt_col='drug_single_str', # 정답셋 항목 (세미콜론으로 구분)
#     pred_col='drugs'          # 예측 항목 (세미콜론으로 구분)
# )

파일의 열 확인

In [14]:
print(ground_truth_file)

NameError: name 'ground_truth_file' is not defined

In [13]:
import pandas as pd
import numpy as np

def parse_and_clean_set(text, delimiter=';'):
    """
    문자열을 세트(Set)로 변환하고, 공백 제거 및 소문자 변환을 수행합니다.
    """
    if pd.isna(text) or text.strip() == '':
        return set()
    
    # 쉼표(,)가 아닌 세미콜론(;)을 구분자로 사용
    items = text.split(delimiter)
    
    # 각 항목의 공백 제거 및 소문자 변환 후 Set 반환
    return set(item.strip().lower() for item in items if item.strip())

def calculate_set_metrics(ground_truth_file, prediction_file, merge_col, gt_col, pred_col):
    """
    두 파일을 병합하고 세트 기반의 Precision, Recall, F1-score를 계산합니다.
    """
    try:
        # 1. 파일 로드
        df_gt = pd.read_csv(ground_truth_file)
        df_pred = pd.read_csv(prediction_file)

        # 2. 공통열(title)로 데이터프레임 병합
        merged_df = pd.merge(df_gt, df_pred, on=merge_col, how='inner')

        if merged_df.empty:
            print("경고: 병합된 데이터가 없습니다. 공통 'title' 열을 확인해 주세요.")
            return None

        # 3. 항목을 세트(Set)로 변환
        merged_df['gt_set'] = merged_df[gt_col].apply(parse_and_clean_set)
        merged_df['pred_set'] = merged_df[pred_col].apply(parse_and_clean_set)

        # 4. 각 행별 TP, FP, FN 계산 함수
        def calculate_row_metrics(row):
            gt = row['gt_set']
            pred = row['pred_set']
            
            # True Positives: 예측과 정답의 교집합
            tp = len(gt.intersection(pred))
            
            # False Positives: 예측은 했지만 정답이 아닌 것 (예측 - 정답)
            fp = len(pred.difference(gt))
            
            # False Negatives: 정답이지만 예측하지 못한 것 (정답 - 예측)
            fn = len(gt.difference(pred))
            
            return pd.Series({'TP': tp, 'FP': fp, 'FN': fn})

        # 5. 모든 행에 걸쳐 TP, FP, FN 계산
        metrics_df = merged_df.apply(calculate_row_metrics, axis=1)
        
        # 6. Micro-averaged 합계 계산
        total_tp = metrics_df['TP'].sum()
        total_fp = metrics_df['FP'].sum()
        total_fn = metrics_df['FN'].sum()

        # 7. Precision, Recall, F1-score 계산
        
        # 분모가 0인 경우를 처리합니다.
        
        # Precision = TP / (TP + FP)
        if (total_tp + total_fp) == 0:
            precision = 0.0
        else:
            precision = total_tp / (total_tp + total_fp)
        
        # Recall = TP / (TP + FN)
        if (total_tp + total_fn) == 0:
            recall = 0.0
        else:
            recall = total_tp / (total_tp + total_fn)
            
        # F1-score = 2 * (P * R) / (P + R)
        if (precision + recall) == 0:
            f1_score = 0.0
        else:
            f1_score = 2 * (precision * recall) / (precision + recall)

        # 8. 결과 출력
        print(f"**총 비교 대상 행 수:** {len(merged_df)}개")
        print("--- 총 누적 합계 (Micro-averaged Basis) ---")
        print(f"True Positives (TP): {total_tp}")
        print(f"False Positives (FP): {total_fp}")
        print(f"False Negatives (FN): {total_fn}")
        print("------------------------------------------")
        print(f"**Precision (정밀도):** {precision:.4f} ({precision * 100:.2f}%)")
        print(f"**Recall (재현율):** {recall:.4f} ({recall * 100:.2f}%)")
        print(f"**F1-score:** {f1_score:.4f} ({f1_score * 100:.2f}%)")

        return {
            'precision': precision, 
            'recall': recall, 
            'f1_score': f1_score
        }

    except FileNotFoundError as e:
        print(f"오류: 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요: {e}")
        return None
    except KeyError as e:
        print(f"오류: 열 이름을 찾을 수 없습니다. 'title', 'drug_single_str', 'drugs' 열 이름을 확인해 주세요: {e}")
        return None

# --- 실행 부분 (사용 예시) ---
# precision_score = calculate_set_metrics(
#     ground_truth_file='ground_truth.csv',
#     prediction_file='my_predictions.csv',
#     merge_col='title',
#     gt_col='drug_single_str', # 정답셋 항목 (세미콜론으로 구분)
#     pred_col='drugs'          # 예측 항목 (세미콜론으로 구분)
# )

In [19]:
# 💡 이곳을 수정하세요!
# ---------------------------------------------
ground_truth_file = gold  # 예: 'ground_truth.csv'
prediction_file = pred # 예: 'my_predictions.csv'
# ---------------------------------------------

metrics = calculate_set_metrics(
    ground_truth_file=ground_truth_file,
    prediction_file=prediction_file,
    merge_col='title',
    gt_col='drug_single_str',
    pred_col='drugs'
)

TypeError: argument of type 'method' is not iterable

In [26]:
# CSV 불러오기
gold = pd.read_csv("gold_single.csv")
pred = pd.read_csv("pred_1.csv")

print(gold.columns)
print(pred.columns)

Index(['title', 'single_drugs_str'], dtype='object')
Index(['drugs', 'title'], dtype='object')


여러개 drugs 정답률 평가

In [27]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# CSV 파일 불러오기
df_truth = pd.read_csv("gold_single.csv", encoding='utf-8')   # 정답셋 (title, drug_single_str 포함)
df_pred  = pd.read_csv("pred_1.csv", encoding='utf-8') # 예측셋 (title, drugs 포함)

# title 기준으로 merge
df = pd.merge(df_truth, df_pred, on="title", how="inner")

# 문자열을 set으로 변환하는 함수 (; 기준 split)
def to_set(x):
    if pd.isna(x):
        return set()
    return set([s.strip().lower() for s in str(x).split(";") if s.strip()])

# 정답과 예측을 set으로 변환
df["truth_set"] = df["single_drugs_str"].apply(to_set)
df["pred_set"]  = df["drugs"].apply(to_set)

# 평가용 리스트
y_true, y_pred = [], []

for truth, pred in zip(df["truth_set"], df["pred_set"]):
    for p in pred:
        y_pred.append(1 if p in truth else 0)
        y_true.append(1 if p in truth else 0)

# precision, recall, f1 계산
precision = precision_score(y_true, y_pred, zero_division=0)
recall    = recall_score(y_true, y_pred, zero_division=0)
f1        = f1_score(y_true, y_pred, zero_division=0)

print("Precision:", round(precision, 3))
print("Recall:", round(recall, 3))
print("F1 Score:", round(f1, 3))


Precision: 1.0
Recall: 1.0
F1 Score: 1.0


코드 다시

In [29]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# CSV 파일 불러오기
df_truth = pd.read_csv("gold_single.csv")   # 정답셋 (title, drug_single_str 포함)
df_pred  = pd.read_csv("pred_1.csv") # 예측셋 (title, drugs 포함)

# title 기준으로 merge
df = pd.merge(df_truth, df_pred, on="title", how="inner")

# 문자열을 set으로 변환 (; 기준 split)
def to_set(x):
    if pd.isna(x):
        return set()
    return set([s.strip().lower() for s in str(x).split(";") if s.strip()])

df["truth_set"] = df["single_drugs_str"].apply(to_set)
df["pred_set"]  = df["drugs"].apply(to_set)

# 전체 데이터에서 등장하는 모든 drug의 집합
all_drugs = sorted(set().union(*df["truth_set"], *df["pred_set"]))

# one-hot encoding
y_true, y_pred = [], []

for truth, pred in zip(df["truth_set"], df["pred_set"]):
    y_true.append([1 if drug in truth else 0 for drug in all_drugs])
    y_pred.append([1 if drug in pred else 0 for drug in all_drugs])

# precision, recall, f1 (micro 평균: 전체 예측 단위로 평가)
precision = precision_score(y_true, y_pred, average="micro", zero_division=0)
recall    = recall_score(y_true, y_pred, average="micro", zero_division=0)
f1        = f1_score(y_true, y_pred, average="micro", zero_division=0)

print("Precision:", round(precision, 3))
print("Recall:", round(recall, 3))
print("F1 Score:", round(f1, 3))


Precision: 0.311
Recall: 0.667
F1 Score: 0.424


- title 마다 정답셋과 예측셋이 얼마나 겹쳤는지 계산 
- per-title accuracy : 겹친 drug 개수/(정답 drug 개수와 예측 drug 개수)

In [30]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# CSV 파일 불러오기
df_truth = pd.read_csv("gold_single.csv")   # 정답셋 (title, drug_single_str 포함)
df_pred  = pd.read_csv("pred_1.csv") # 예측셋 (title, drugs 포함)

# title 기준으로 merge
df = pd.merge(df_truth, df_pred, on="title", how="inner")

# 문자열을 set으로 변환 (; 기준 split)
def to_set(x):
    if pd.isna(x):
        return set()
    return set([s.strip().lower() for s in str(x).split(";") if s.strip()])

df["truth_set"] = df["single_drugs_str"].apply(to_set)
df["pred_set"]  = df["drugs"].apply(to_set)

# ✅ Title별 Accuracy 계산 (Jaccard 유사도)
def jaccard(truth, pred):
    if not truth and not pred:
        return 1.0
    return len(truth & pred) / len(truth | pred)

df["title_accuracy"] = df.apply(lambda row: jaccard(row["truth_set"], row["pred_set"]), axis=1)

print("🔹 Title별 Accuracy:")
print(df[["title", "title_accuracy"]].head())  # 앞부분만 출력

# ✅ 전체 precision/recall/f1 계산 (multi-label 방식)
all_drugs = sorted(set().union(*df["truth_set"], *df["pred_set"]))

y_true, y_pred = [], []
for truth, pred in zip(df["truth_set"], df["pred_set"]):
    y_true.append([1 if drug in truth else 0 for drug in all_drugs])
    y_pred.append([1 if drug in pred else 0 for drug in all_drugs])

precision = precision_score(y_true, y_pred, average="micro", zero_division=0)
recall    = recall_score(y_true, y_pred, average="micro", zero_division=0)
f1        = f1_score(y_true, y_pred, average="micro", zero_division=0)

print("\n🔹 전체 평가 지표:")
print("Precision:", round(precision, 3))
print("Recall:", round(recall, 3))
print("F1 Score:", round(f1, 3))


🔹 Title별 Accuracy:
                                               title  title_accuracy
0  Conditionally reprogrammed colorectal cancer c...        0.000000
1  Activation of the mTOR Pathway by Oxaliplatin ...        0.125000
2  Integrative In Vivo Drug Testing Using Gene Ex...        0.714286
3  Dual Fatty Acid Synthase and HER2 Signaling Bl...        0.076923
4  Acquired resistance to trastuzumab/pertuzumab ...        0.600000

🔹 전체 평가 지표:
Precision: 0.311
Recall: 0.667
F1 Score: 0.424


title별 accuracy와 에러 케이스 같이 보는 코드

In [31]:
# title별 accuracy, 정답셋, 예측셋 같이 보기
result_df = df[["title", "truth_set", "pred_set", "title_accuracy"]].copy()

# accuracy 낮은 순으로 정렬
result_df = result_df.sort_values(by="title_accuracy")

print(result_df.head(20))  # accuracy 낮은 상위 20개 확인


                                               title  \
0  Conditionally reprogrammed colorectal cancer c...   
3  Dual Fatty Acid Synthase and HER2 Signaling Bl...   
1  Activation of the mTOR Pathway by Oxaliplatin ...   
5  CDK4/6 Inhibitors Impede Chemoresistance and I...   
4  Acquired resistance to trastuzumab/pertuzumab ...   
6  Exploration of Immune-Modulatory Effects of Am...   
2  Integrative In Vivo Drug Testing Using Gene Ex...   

                                           truth_set  \
0    {cetuximab, sorafenib, trametinib, palbociclib}   
3                               {egcg, temsirolimus}   
1                          {everolimus, oxaliplatin}   
5                             {cisplatin, etoposide}   
4                 {tas0728, trastuzumab, pertuzumab}   
6                       {pembrolizumab, amivantamab}   
2  {bevacizumab, neratinib, docetaxel, sorafenib,...   

                                            pred_set  title_accuracy  
0  {crizotinib, braf inhibitor,

방법: title_accuracy == 0.0 인 케이스 제거

In [32]:
# title별 accuracy 계산 (앞에서 만든 df 활용)
valid_df = df[df["title_accuracy"] > 0].copy()

print(f"전체 데이터 개수: {len(df)}")
print(f"제외 후 남은 데이터 개수: {len(valid_df)}")

# 다시 precision/recall/f1 계산
y_true, y_pred = [], []
all_drugs = sorted(set().union(*valid_df["truth_set"], *valid_df["pred_set"]))

for truth, pred in zip(valid_df["truth_set"], valid_df["pred_set"]):
    y_true.append([1 if drug in truth else 0 for drug in all_drugs])
    y_pred.append([1 if drug in pred else 0 for drug in all_drugs])

from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y_true, y_pred, average="micro", zero_division=0)
recall    = recall_score(y_true, y_pred, average="micro", zero_division=0)
f1        = f1_score(y_true, y_pred, average="micro", zero_division=0)

print("\n🔹 title_accuracy > 0.0 케이스만 평가")
print("Precision:", round(precision, 3))
print("Recall:", round(recall, 3))
print("F1 Score:", round(f1, 3))


전체 데이터 개수: 7
제외 후 남은 데이터 개수: 6

🔹 title_accuracy > 0.0 케이스만 평가
Precision: 0.368
Recall: 0.824
F1 Score: 0.509


예측셋에 있는 title 중심으로 비교하기

In [33]:
# 예측셋 title만 남기기
df_truth_filtered = df_truth[df_truth["title"].isin(df_pred["title"])]

# 다시 merge
df = pd.merge(df_truth_filtered, df_pred, on="title", how="inner")

print("정답셋 개수:", len(df_truth))
print("예측셋 개수:", len(df_pred))
print("필터링 후 남은 title 개수:", len(df))


정답셋 개수: 163
예측셋 개수: 129
필터링 후 남은 title 개수: 7


예측셋(24개) 중심으로 다시

In [36]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# CSV 파일 불러오기
df_truth = pd.read_csv("gold_single.csv")   # 정답셋 (title, drug_single_str 포함)
df_pred  = pd.read_csv("pred_1.csv")        # 예측셋 (title, drugs 포함)

# title 문자열 정리 (대소문자/공백 문제 방지)
df_truth["title"] = df_truth["title"].astype(str).str.strip().str.lower()
df_pred["title"]  = df_pred["title"].astype(str).str.strip().str.lower()

# 정답셋을 예측셋에 맞춰 필터링
df_truth_filtered = df_truth[df_truth["title"].isin(df_pred["title"])]

# title 기준 merge
df = pd.merge(df_truth_filtered, df_pred, on="title", how="inner")

print("정답셋 전체 개수:", len(df_truth))
print("예측셋 전체 개수:", len(df_pred))
print("매칭된 title 개수:", len(df))

# 문자열을 set으로 변환 (; 기준 split)
def to_set(x):
    if pd.isna(x):
        return set()
    return set([s.strip().lower() for s in str(x).split(";") if s.strip()])

df["truth_set"] = df["single_drugs_str"].apply(to_set)
df["pred_set"]  = df["drugs"].apply(to_set)

# title별 accuracy (Jaccard 유사도)
def jaccard(truth, pred):
    if not truth and not pred:
        return 1.0
    return len(truth & pred) / len(truth | pred)

df["title_accuracy"] = df.apply(lambda row: jaccard(row["truth_set"], row["pred_set"]), axis=1)

# 전체 평가 지표 (multi-label 방식)
all_drugs = sorted(set().union(*df["truth_set"], *df["pred_set"]))
y_true, y_pred = [], []
for truth, pred in zip(df["truth_set"], df["pred_set"]):
    y_true.append([1 if drug in truth else 0 for drug in all_drugs])
    y_pred.append([1 if drug in pred else 0 for drug in all_drugs])

precision = precision_score(y_true, y_pred, average="micro", zero_division=0)
recall    = recall_score(y_true, y_pred, average="micro", zero_division=0)
f1        = f1_score(y_true, y_pred, average="micro", zero_division=0)

print("\n🔹 title별 accuracy 예시:")
print(df[["title", "truth_set", "pred_set", "title_accuracy"]].head())

print("\n🔹 전체 평가 지표:")
print("Precision:", round(precision, 3))
print("Recall:", round(recall, 3))
print("F1 Score:", round(f1, 3))


정답셋 전체 개수: 163
예측셋 전체 개수: 129
매칭된 title 개수: 8

🔹 title별 accuracy 예시:
                                               title  \
0  conditionally reprogrammed colorectal cancer c...   
1  activation of the mtor pathway by oxaliplatin ...   
2  integrative in vivo drug testing using gene ex...   
3  dual fatty acid synthase and her2 signaling bl...   
4  acquired resistance to trastuzumab/pertuzumab ...   

                                           truth_set  \
0    {cetuximab, sorafenib, trametinib, palbociclib}   
1                          {everolimus, oxaliplatin}   
2  {bevacizumab, neratinib, docetaxel, sorafenib,...   
3                               {egcg, temsirolimus}   
4                 {tas0728, trastuzumab, pertuzumab}   

                                            pred_set  title_accuracy  
0  {crizotinib, braf inhibitor, docetaxel, cdk4/6...        0.000000  
1  {oxaliplatin, cetuximab, panitumumab, bevacizu...        0.125000  
2  {bevacizumab, neratinib, docetaxel, soraf

In [44]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# ----------------- 파일 로드 및 매칭 확인 -----------------
# CSV 파일 불러오기
df_truth = pd.read_csv("gold_single.csv")    # 정답셋 (title, drug_single_str 포함)
df_pred  = pd.read_csv("pred_1.csv")         # 예측셋 (title, drugs 포함)

# title 문자열 정리 (대소문자/공백 문제 방지)
df_truth["title"] = df_truth["title"].astype(str).str.strip().str.lower()
df_pred["title"]  = df_pred["title"].astype(str).str.strip().str.lower()

# 예측셋에 있는 title만 정답셋에서 필터링
df_truth_filtered = df_truth[df_truth["title"].isin(df_pred["title"])]

# title 기준 merge (Inner Join)
# 매칭되는 title을 가진 행만 남습니다.
df = pd.merge(df_truth_filtered, df_pred, on="title", how="inner")

print("정답셋 전체 개수:", len(df_truth))
print("예측셋 전체 개수:", len(df_pred))
print("✅ 매칭된 title 개수:", len(df))

# ----------------- 지표 계산 -----------------
# 문자열을 set으로 변환 (; 기준 split)
def to_set(x):
    if pd.isna(x):
        return set()
    # 주의: 여기서 사용된 컬럼명은 사용자님이 제공하신 코드의 컬럼명입니다.
    # df["single_drugs_str"]과 df["drugs"]를 사용합니다.
    return set([s.strip().lower() for s in str(x).split(";") if s.strip()])

# 컬럼명 확인: 원본 코드의 컬럼명인 "single_drugs_str"을 사용합니다.
# 만약 정답셋의 컬럼명이 "drug_single_str"이라면 'single_drugs_str'을 'drug_single_str'로 수정해야 합니다.
df["truth_set"] = df["single_drugs_str"].apply(to_set)
df["pred_set"]  = df["drugs"].apply(to_set)

# title별 accuracy (Jaccard 유사도)
def jaccard(truth, pred):
    if not truth and not pred:
        return 1.0
    return len(truth & pred) / len(truth | pred)

df["title_accuracy"] = df.apply(lambda row: jaccard(row["truth_set"], row["pred_set"]), axis=1)

# 전체 평가 지표 (multi-label Micro-averaged 방식)
# 1. 정답과 예측에 있는 모든 고유한 약물 목록을 만듭니다.
all_drugs = sorted(set().union(*df["truth_set"], *df["pred_set"]))
y_true, y_pred = [], []

# 2. 각 행의 정답/예측을 전체 약물 목록 기준으로 0과 1의 벡터로 변환합니다.
for truth, pred in zip(df["truth_set"], df["pred_set"]):
    y_true.append([1 if drug in truth else 0 for drug in all_drugs])
    y_pred.append([1 if drug in pred else 0 for drug in all_drugs])

# 3. sklearn 함수로 Micro-averaged 지표를 계산합니다.
precision = precision_score(y_true, y_pred, average="micro", zero_division=0)
recall    = recall_score(y_true, y_pred, average="micro", zero_division=0)
f1        = f1_score(y_true, y_pred, average="micro", zero_division=0)

# ----------------- 최종 결과 출력 -----------------
print("\n🔹 title별 accuracy 예시:")
print(df[["title", "truth_set", "pred_set", "title_accuracy"]].head())

print("\n🔹 전체 평가 지표 (Micro-averaged):")
print(f"Precision: {round(precision, 3)}")
print(f"Recall: {round(recall, 3)}")
print(f"F1 Score: {round(f1, 3)}")

정답셋 전체 개수: 163
예측셋 전체 개수: 129
✅ 매칭된 title 개수: 8

🔹 title별 accuracy 예시:
                                               title  \
0  conditionally reprogrammed colorectal cancer c...   
1  activation of the mtor pathway by oxaliplatin ...   
2  integrative in vivo drug testing using gene ex...   
3  dual fatty acid synthase and her2 signaling bl...   
4  acquired resistance to trastuzumab/pertuzumab ...   

                                           truth_set  \
0    {cetuximab, sorafenib, trametinib, palbociclib}   
1                          {everolimus, oxaliplatin}   
2  {bevacizumab, neratinib, docetaxel, sorafenib,...   
3                               {egcg, temsirolimus}   
4                 {tas0728, trastuzumab, pertuzumab}   

                                            pred_set  title_accuracy  
0  {crizotinib, braf inhibitor, docetaxel, cdk4/6...        0.000000  
1  {oxaliplatin, cetuximab, panitumumab, bevacizu...        0.125000  
2  {bevacizumab, neratinib, docetaxel, sor

In [52]:

# ----------------- 파일 로드 및 매칭 확인 -----------------
# CSV 파일 불러오기
df_truth = pd.read_csv("gold_single.csv")    # 정답셋 (title, single_drugs_str 포함)
df_pred  = pd.read_csv("pred_1.csv")         # 예측셋 (title, drugs 포함)

# title 문자열 정리 (대소문자/공백 문제 방지)
# df_truth["title"] = df_truth["title"].astype(str).str.strip().str.lower()
# df_pred["title"]  = df_pred["title"].astype(str).str.strip().str.lower()

# 예측셋에 있는 title만 정답셋에서 필터링
# df_truth_filtered = df_truth[df_truth["title"].isin(df_pred["title"])]

# title 기준 merge (Inner Join)
# 매칭되는 title을 가진 행만 남습니다.
df = pd.merge(df_truth_filtered, df_pred, on="title", how="inner")

print("정답셋 전체 개수:", len(df_truth))
print("예측셋 전체 개수:", len(df_pred))
print("✅ 매칭된 title 개수:", len(df))

정답셋 전체 개수: 163
예측셋 전체 개수: 129
✅ 매칭된 title 개수: 7


In [54]:
import pandas as pd

# 1. CSV 파일 불러오기 (파일명은 사용자님의 파일명과 동일하다고 가정)
df_truth = pd.read_csv("gold_single.csv")    # 정답셋
df_pred  = pd.read_csv("pred_1.csv")         # 예측셋

# 2. title 문자열 정리 (매칭 오류 방지)
df_truth["title"] = df_truth["title"].astype(str).str.strip().str.lower()
df_pred["title"]  = df_pred["title"].astype(str).str.strip().str.lower()

# 3. 예측셋(df_pred)에 있는 title만 정답셋(df_truth)에서 필터링
# 예측셋을 평가하는 것이 목적이므로, 예측셋에 없는 title은 평가 대상에서 제외됩니다.
df_truth_filtered = df_truth[df_truth["title"].isin(df_pred["title"])]

# 4. title 기준 merge (Inner Join)
# 예측셋의 title과 정답셋의 title이 일치하는 행만 최종 데이터프레임 `df`에 남습니다.
df = pd.merge(df_truth_filtered, df_pred, on="title", how="inner")

print("------------------------------------------")
print(f"정답셋 전체 개수: {len(df_truth)}개")
print(f"예측셋 전체 개수: {len(df_pred)}개")
print(f"✅ 예측셋 중심으로 매칭된 title 개수: {len(df)}개")
print("------------------------------------------")

# 매칭된 데이터프레임의 상위 5개를 확인합니다. (컬럼명 확인용)
print("매칭 결과 데이터프레임 (df) 상위 5행:")
# print(df.head())
print(df)

------------------------------------------
정답셋 전체 개수: 163개
예측셋 전체 개수: 129개
✅ 예측셋 중심으로 매칭된 title 개수: 8개
------------------------------------------
매칭 결과 데이터프레임 (df) 상위 5행:
                                               title  \
0  conditionally reprogrammed colorectal cancer c...   
1  activation of the mtor pathway by oxaliplatin ...   
2  integrative in vivo drug testing using gene ex...   
3  dual fatty acid synthase and her2 signaling bl...   
4  acquired resistance to trastuzumab/pertuzumab ...   
5  identification of src family kinases as potent...   
6  cdk4/6 inhibitors impede chemoresistance and i...   
7  exploration of immune-modulatory effects of am...   

                                    single_drugs_str  \
0      Cetuximab; Palbociclib; Sorafenib; Trametinib   
1                            Everolimus; Oxaliplatin   
2  Bevacizumab; Docetaxel; Everolimus; Neratinib;...   
3                                 EGCG; Temsirolimus   
4                   TAS0728; Trastuzumab; pe

- 유사도 매칭으로 논문title 매칭시키기

In [55]:
pip install fuzzywuzzy python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 11.1 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [python-Levenshtein]
Note: you may need to restart the kernel to use updated packages.


In [57]:
import pandas as pd
from fuzzywuzzy import fuzz # 문자열 유사도 계산 라이브러리

# ----------------- 설정 값 -----------------
# title이 최소한 이 점수 이상일 때만 매칭으로 간주합니다.
# 85~90점 정도가 일반적인 추천 값입니다.
SIMILARITY_THRESHOLD = 85 
# ------------------------------------------

# 1. 파일 로드 및 title 정리 (이전과 동일)
df_truth = pd.read_csv("gold_single.csv")
df_pred  = pd.read_csv("pred_1.csv")

df_truth["title_clean"] = df_truth["title"].astype(str).str.strip().str.lower()
df_pred["title_clean"]  = df_pred["title"].astype(str).str.strip().str.lower()

# 2. title 매칭 함수 정의 (Fuzzy Matching)
def find_best_match(target_title, choices):
    """
    하나의 target_title에 대해 choices 목록에서 가장 유사한 title을 찾습니다.
    """
    best_score = 0
    best_match = None
    
    # choices는 정답셋의 title 목록입니다.
    for choice in choices:
        # fuzz.ratio: 두 문자열이 얼마나 유사한지 0~100점 사이의 점수를 반환
        score = fuzz.ratio(target_title, choice)
        if score > best_score:
            best_score = score
            best_match = choice
    
    # 설정된 임계치(SIMILARITY_THRESHOLD)를 넘어야만 매칭으로 인정
    if best_score >= SIMILARITY_THRESHOLD:
        return best_match
    else:
        return None # 임계치를 넘지 못하면 매칭 실패

# 3. 정답셋의 title 목록을 리스트로 준비
truth_titles = df_truth["title_clean"].tolist()

# 4. 예측셋의 각 title에 대해 가장 유사한 정답 title 찾기
df_pred['matched_title_clean'] = df_pred['title_clean'].apply(
    lambda x: find_best_match(x, truth_titles)
)
# 매칭 점수도 확인하고 싶다면 아래 주석 해제:
# df_pred['match_score'] = df_pred.apply(
#     lambda row: fuzz.ratio(row['title_clean'], row['matched_title_clean']) if row['matched_title_clean'] else 0, axis=1
# )

# 5. 매칭된 행만 필터링 (matched_title_clean이 None이 아닌 행)
df_matched = df_pred.dropna(subset=['matched_title_clean']).copy()

# 6. 정답 데이터프레임과 병합
# 매칭된 title (matched_title_clean)을 기준으로 정답셋의 title_clean과 병합합니다.
df = pd.merge(
    df_matched,
    df_truth[['title_clean', 'single_drugs_str']], # 정답셋에서 필요한 컬럼만 선택
    left_on='matched_title_clean', 
    right_on='title_clean', 
    how='inner',
    suffixes=('_pred', '_truth') # 컬럼명 충돌 방지
)

# 최종 결과 정리
print("------------------------------------------")
print(f"정답셋 전체 개수: {len(df_truth)}개")
print(f"예측셋 전체 개수: {len(df_pred)}개")
print(f"✅ Fuzzy Matching 후 매칭된 title 개수: {len(df)}개")
print("------------------------------------------")

# 매칭 결과 확인 (원본 title과 매칭된 title 비교)
print("Fuzzy Matching 결과 상위 5행 (title 비교):")
print(df)

------------------------------------------
정답셋 전체 개수: 163개
예측셋 전체 개수: 129개
✅ Fuzzy Matching 후 매칭된 title 개수: 23개
------------------------------------------
Fuzzy Matching 결과 상위 5행 (title 비교):
                                                drugs  \
0        cisplatin;etoposide;CDK4/6 inhibitors;PD;LEE   
1   Crizotinib;5‑Fluorouracil (5‑FU);Docetaxel;EGF...   
2                       KRT‑232;Trametinib;Navitoclax   
3                                             BGB‑283   
4      TAS0728;Trastuzumab;Pertuzumab;T‑DM1;Lapatinib   
5   sorafenib;everolimus;docetaxel;bevacizumab;PX‑...   
6   rapamycin;AZD8055;AZD2014;5‑fluorouracil (5FU)...   
7                    cetuximab;carboplatin;Paraplatin   
8   carboplatin;paclitaxel;pemetrexed;cisplatin;et...   
9   carboplatin;paclitaxel;pemetrexed;cisplatin;et...   
10                           osimertinib;capivasertib   
11  Letrozole;Tamoxifen;Fulvestrant;Lasofoxifene;E...   
12                                 pemetrexed;PP1;PP2   
13  EGFR in

유사도 점수 나오게

In [61]:
import pandas as pd
from fuzzywuzzy import fuzz

# ----------------- 설정 값 -----------------
SIMILARITY_THRESHOLD = 85 # 이 점수 이상일 때만 매칭으로 간주
# ------------------------------------------

# 1. 파일 로드 및 title 정리 (이전과 동일)
df_truth = pd.read_csv("gold_single.csv")
df_pred  = pd.read_csv("pred_1.csv")

df_truth["title_clean"] = df_truth["title"].astype(str).str.strip().str.lower()
df_pred["title_clean"]  = df_pred["title"].astype(str).str.strip().str.lower()

print(df_truth.columns)



Index(['title', 'single_drugs_str', 'title_clean'], dtype='object')


In [62]:
print(df_pred.columns)

Index(['drugs', 'title', 'title_clean'], dtype='object')


In [63]:
print(df_truth)

                                                 title  \
0    Conditionally reprogrammed colorectal cancer c...   
1    Combining MEK and SRC inhibitors for treatment...   
2    Sensitivity of KRAS-Mutant Colorectal Cancers ...   
3    Targeting RAS Mutant Colorectal Cancer with Du...   
4    Mcl-1 inhibition overcomes intrinsic and acqui...   
..                                                 ...   
158  LKB1 regulates JNK-dependent stress signaling ...   
159  Antitumor efficacy of XPO1 inhibitor Selinexor...   
160  Signature-driven repurposing of Midostaurin fo...   
161  Costunolide is a dual inhibitor of MEK1 and AK...   
162  KRT-232 and navitoclax enhance trametinib’s an...   

                                  single_drugs_str  \
0    Cetuximab; Palbociclib; Sorafenib; Trametinib   
1                            Dasatinib; Trametinib   
2                          Palbociclib; Trametinib   
3                          Palbociclib; Trametinib   
4                              Re

In [64]:
print(df_pred)

                                                 drugs  \
0         cisplatin;etoposide;CDK4/6 inhibitors;PD;LEE   
1    Crizotinib;5‑Fluorouracil (5‑FU);Docetaxel;EGF...   
2                        KRT‑232;Trametinib;Navitoclax   
3                                              BGB‑283   
4       TAS0728;Trastuzumab;Pertuzumab;T‑DM1;Lapatinib   
..                                                 ...   
124                                                NaN   
125                                                NaN   
126                                                NaN   
127                                                NaN   
128                                                NaN   

                                                 title  \
0    CDK4/6 Inhibitors Impede Chemoresistance and I...   
1    Conditionally reprogrammed colorectal cancer c...   
2    KRT-232 and navitoclax enhance trametinib’s  \...   
3    BGB-283, a Novel RAF Kinase and EGFR Inhibitor...   
4    Acquired

In [66]:
import pandas as pd
from fuzzywuzzy import fuzz

# ----------------- 설정 값 -----------------
SIMILARITY_THRESHOLD = 90 # 이 점수 이상일 때만 매칭으로 간주
# ------------------------------------------

# 1. 파일 로드 및 title 정리
df_truth = pd.read_csv("gold_single.csv")
df_pred  = pd.read_csv("pred_1.csv")

# 🚨 추가된 코드: df_pred에서 title이 NaN인 행 제거
df_pred = df_pred.dropna(subset=['title'])

df_truth["title_clean"] = df_truth["title"].astype(str).str.strip().str.lower()
df_pred["title_clean"]  = df_pred["title"].astype(str).str.strip().str.lower()

# 2. title 매칭 함수 수정 (Fuzzy Matching + Score 반환)
def find_best_match_with_score(target_title, choices):
    """
    가장 유사한 title과 그 유사도 점수를 튜플로 반환합니다.
    """
    best_score = 0
    best_match = None
    
    for choice in choices:
        score = fuzz.ratio(target_title, choice)
        if score > best_score:
            best_score = score
            best_match = choice
    
    # 설정된 임계치(SIMILARITY_THRESHOLD)를 넘어야만 매칭으로 인정
    if best_score >= SIMILARITY_THRESHOLD:
        return best_match, best_score
    else:
        return None, 0

# 3. 정답셋의 title 목록을 리스트로 준비
truth_titles = df_truth["title_clean"].tolist()

# 4. 예측셋의 각 title에 대해 가장 유사한 정답 title 찾기 및 점수 저장
results = df_pred['title_clean'].apply(
    lambda x: find_best_match_with_score(x, truth_titles)
)

# 5. 결과를 DataFrame의 새 열로 분리
df_pred[['matched_title_clean', 'match_score']] = pd.DataFrame(
    results.tolist(), index=df_pred.index
)

# 6. 매칭된 행만 필터링 (matched_title_clean이 None이 아닌 행)
df_matched = df_pred.dropna(subset=['matched_title_clean']).copy()

# 7. 정답 데이터프레임과 병합
df = pd.merge(
    df_matched,
    df_truth[['title_clean', 'single_drugs_str']], 
    left_on='matched_title_clean', 
    right_on='title_clean', 
    how='inner',
    suffixes=('_pred', '_truth') 
)

# 8. 최종 결과 출력 (요청하신 형식)
print("------------------------------------------")
print(f"정답셋 전체 개수 (NaN 제거 전): {len(df_truth)}개")
print(f"예측셋 전체 개수 (NaN 제거 후): {len(df_pred)}개")
print(f"✅ Fuzzy Matching 후 매칭된 title 개수: {len(df)}개")
print("------------------------------------------")

print("Fuzzy Matching 결과 상위 10행 (title 및 점수 비교):")

output_cols = [
    'title_pred',         # 예측셋의 원본 title
    'title_truth',        # 매칭된 정답셋의 title
    'match_score',        # 유사도 점수 (0~100)
    'drugs',              # 예측된 약물 목록 (참고용)
    'single_drugs_str'    # 정답 약물 목록 (참고용)
]

print(df)

------------------------------------------
정답셋 전체 개수 (NaN 제거 전): 163개
예측셋 전체 개수 (NaN 제거 후): 23개
✅ Fuzzy Matching 후 매칭된 title 개수: 23개
------------------------------------------
Fuzzy Matching 결과 상위 10행 (title 및 점수 비교):
                                                drugs  \
0        cisplatin;etoposide;CDK4/6 inhibitors;PD;LEE   
1   Crizotinib;5‑Fluorouracil (5‑FU);Docetaxel;EGF...   
2                       KRT‑232;Trametinib;Navitoclax   
3                                             BGB‑283   
4      TAS0728;Trastuzumab;Pertuzumab;T‑DM1;Lapatinib   
5   sorafenib;everolimus;docetaxel;bevacizumab;PX‑...   
6   rapamycin;AZD8055;AZD2014;5‑fluorouracil (5FU)...   
7                    cetuximab;carboplatin;Paraplatin   
8   carboplatin;paclitaxel;pemetrexed;cisplatin;et...   
9   carboplatin;paclitaxel;pemetrexed;cisplatin;et...   
10                           osimertinib;capivasertib   
11  Letrozole;Tamoxifen;Fulvestrant;Lasofoxifene;E...   
12                                 pemetr

정답 일치도 계산

In [68]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# ----------------- 1. 함수 정의 -----------------
def to_set(x):
    """
    세미콜론(;)으로 구분된 문자열을 클리닝된 세트(Set)로 변환합니다.
    """
    if pd.isna(x):
        return set()
    return set([s.strip().lower() for s in str(x).split(";") if s.strip()])

# ----------------- 2. 약물 목록을 Set으로 변환 -----------------
# 이전 단계에서 생성된 데이터프레임 'df'에 대해 작업합니다.
df["truth_set"] = df["single_drugs_str"].apply(to_set)
df["pred_set"]  = df["drugs"].apply(to_set)

# ----------------- 3. 평가 지표 계산 -----------------
# 전체 데이터셋에 등장하는 모든 고유한 약물을 모아 평가를 위한 '사전'을 만듭니다.
all_drugs = sorted(set().union(*df["truth_set"], *df["pred_set"]))

# 각 행의 정답/예측을 0과 1로 구성된 벡터로 변환합니다.
y_true, y_pred = [], []
for truth, pred in zip(df["truth_set"], df["pred_set"]):
    y_true.append([1 if drug in truth else 0 for drug in all_drugs])
    y_pred.append([1 if drug in pred else 0 for drug in all_drugs])

# sklearn 함수로 Micro-averaged 지표를 계산합니다.
precision = precision_score(y_true, y_pred, average="micro", zero_division=0)
recall    = recall_score(y_true, y_pred, average="micro", zero_division=0)
f1        = f1_score(y_true, y_pred, average="micro", zero_division=0)

# ----------------- 4. 결과 출력 -----------------
print("--------------------------------------------------")
print("📊 Fuzzy Matched 데이터에 대한 약물 일치도 결과")
print("--------------------------------------------------")
print(f"**Precision (정밀도):** {precision:.3f}")
print(f"**Recall (재현율):** {recall:.3f}")
print(f"**F1 Score (조화 평균):** {f1:.3f}")
print("--------------------------------------------------")

--------------------------------------------------
📊 Fuzzy Matched 데이터에 대한 약물 일치도 결과
--------------------------------------------------
**Precision (정밀도):** 0.320
**Recall (재현율):** 0.600
**F1 Score (조화 평균):** 0.417
--------------------------------------------------


브랜드 네임 뺀 정답셋 비교

In [69]:
import pandas as pd
from fuzzywuzzy import fuzz

# ----------------- 설정 값 -----------------
SIMILARITY_THRESHOLD = 90 # 이 점수 이상일 때만 매칭으로 간주
# ------------------------------------------

# 1. 파일 로드 및 title 정리
df_truth = pd.read_csv("gold_single.csv")
df_pred  = pd.read_csv("pred_2.csv")

# 🚨 추가된 코드: df_pred에서 title이 NaN인 행 제거
df_pred = df_pred.dropna(subset=['title'])

df_truth["title_clean"] = df_truth["title"].astype(str).str.strip().str.lower()
df_pred["title_clean"]  = df_pred["title"].astype(str).str.strip().str.lower()

# 2. title 매칭 함수 수정 (Fuzzy Matching + Score 반환)
def find_best_match_with_score(target_title, choices):
    """
    가장 유사한 title과 그 유사도 점수를 튜플로 반환합니다.
    """
    best_score = 0
    best_match = None
    
    for choice in choices:
        score = fuzz.ratio(target_title, choice)
        if score > best_score:
            best_score = score
            best_match = choice
    
    # 설정된 임계치(SIMILARITY_THRESHOLD)를 넘어야만 매칭으로 인정
    if best_score >= SIMILARITY_THRESHOLD:
        return best_match, best_score
    else:
        return None, 0

# 3. 정답셋의 title 목록을 리스트로 준비
truth_titles = df_truth["title_clean"].tolist()

# 4. 예측셋의 각 title에 대해 가장 유사한 정답 title 찾기 및 점수 저장
results = df_pred['title_clean'].apply(
    lambda x: find_best_match_with_score(x, truth_titles)
)

# 5. 결과를 DataFrame의 새 열로 분리
df_pred[['matched_title_clean', 'match_score']] = pd.DataFrame(
    results.tolist(), index=df_pred.index
)

# 6. 매칭된 행만 필터링 (matched_title_clean이 None이 아닌 행)
df_matched = df_pred.dropna(subset=['matched_title_clean']).copy()

# 7. 정답 데이터프레임과 병합
df = pd.merge(
    df_matched,
    df_truth[['title_clean', 'single_drugs_str']], 
    left_on='matched_title_clean', 
    right_on='title_clean', 
    how='inner',
    suffixes=('_pred', '_truth') 
)

# 8. 최종 결과 출력 (요청하신 형식)
print("------------------------------------------")
print(f"정답셋 전체 개수 (NaN 제거 전): {len(df_truth)}개")
print(f"예측셋 전체 개수 (NaN 제거 후): {len(df_pred)}개")
print(f"✅ Fuzzy Matching 후 매칭된 title 개수: {len(df)}개")
print("------------------------------------------")

print("Fuzzy Matching 결과 상위 10행 (title 및 점수 비교):")

output_cols = [
    'title_pred',         # 예측셋의 원본 title
    'title_truth',        # 매칭된 정답셋의 title
    'match_score',        # 유사도 점수 (0~100)
    'drugs',              # 예측된 약물 목록 (참고용)
    'single_drugs_str'    # 정답 약물 목록 (참고용)
]

print(df)

------------------------------------------
정답셋 전체 개수 (NaN 제거 전): 163개
예측셋 전체 개수 (NaN 제거 후): 23개
✅ Fuzzy Matching 후 매칭된 title 개수: 23개
------------------------------------------
Fuzzy Matching 결과 상위 10행 (title 및 점수 비교):
                                                drugs  \
0              cisplatin;etoposide;CDK4/6 inhibitors;   
1   Crizotinib;5‑Fluorouracil (5‑FU);Docetaxel;EGF...   
2                       KRT‑232;Trametinib;Navitoclax   
3                                             BGB‑283   
4      TAS0728;Trastuzumab;Pertuzumab;T‑DM1;Lapatinib   
5   sorafenib;everolimus;docetaxel;bevacizumab;PX‑...   
6   rapamycin;AZD8055;AZD2014;5‑fluorouracil (5FU)...   
7                    cetuximab;carboplatin;Paraplatin   
8   carboplatin;paclitaxel;pemetrexed;cisplatin;et...   
9   carboplatin;paclitaxel;pemetrexed;cisplatin;et...   
10                           osimertinib;capivasertib   
11  Letrozole;Tamoxifen;Fulvestrant;Lasofoxifene;E...   
12                                       

In [70]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# ----------------- 1. 함수 정의 -----------------
def to_set(x):
    """
    세미콜론(;)으로 구분된 문자열을 클리닝된 세트(Set)로 변환합니다.
    """
    if pd.isna(x):
        return set()
    return set([s.strip().lower() for s in str(x).split(";") if s.strip()])

# ----------------- 2. 약물 목록을 Set으로 변환 -----------------
# 이전 단계에서 생성된 데이터프레임 'df'에 대해 작업합니다.
df["truth_set"] = df["single_drugs_str"].apply(to_set)
df["pred_set"]  = df["drugs"].apply(to_set)

# ----------------- 3. 평가 지표 계산 -----------------
# 전체 데이터셋에 등장하는 모든 고유한 약물을 모아 평가를 위한 '사전'을 만듭니다.
all_drugs = sorted(set().union(*df["truth_set"], *df["pred_set"]))

# 각 행의 정답/예측을 0과 1로 구성된 벡터로 변환합니다.
y_true, y_pred = [], []
for truth, pred in zip(df["truth_set"], df["pred_set"]):
    y_true.append([1 if drug in truth else 0 for drug in all_drugs])
    y_pred.append([1 if drug in pred else 0 for drug in all_drugs])

# sklearn 함수로 Micro-averaged 지표를 계산합니다.
precision = precision_score(y_true, y_pred, average="micro", zero_division=0)
recall    = recall_score(y_true, y_pred, average="micro", zero_division=0)
f1        = f1_score(y_true, y_pred, average="micro", zero_division=0)

# ----------------- 4. 결과 출력 -----------------
print("--------------------------------------------------")
print("📊 Fuzzy Matched 데이터에 대한 약물 일치도 결과")
print("--------------------------------------------------")
print(f"**Precision (정밀도):** {precision:.3f}")
print(f"**Recall (재현율):** {recall:.3f}")
print(f"**F1 Score (조화 평균):** {f1:.3f}")
print("--------------------------------------------------")

--------------------------------------------------
📊 Fuzzy Matched 데이터에 대한 약물 일치도 결과
--------------------------------------------------
**Precision (정밀도):** 0.331
**Recall (재현율):** 0.600
**F1 Score (조화 평균):** 0.426
--------------------------------------------------
